In [1]:
# Instalando las paqueterías necesarias.
%pip install pandas openpyxl requests beautifulsoup4
%pip install wget
%pip install gdown
%pip install unidecode

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Las siguientes librerías nos ayudaran para el desarrollo del proceso de descarga y generación del archivo txt. es necesario que este al principio para su correcto funcionamiento.

Se utiliza librerías especiales como gwdown para descarga de archivos grandes, eliminando la confirmación de permiso de descarga.

Se utiliza librería unicode, para poder quitar los acentos de ciertas palabras y usarlo como nombre de archivos a descargar.


In [2]:
# Importación de librerías para usarlas en el proyecto de descarga y generación del txt con la descripción de esta
import pandas as pd
import wget
import gdown
import requests
import re
import os
from bs4 import BeautifulSoup
from datetime import datetime
from logging import exception
from unidecode import unidecode
from datetime import datetime

La inicialización de ciertas variables ayudaran a preparar la información inicial de cada una de ellas, además de que puedan ser usadas en cualquier parte del código o proceso, sin en el que perjudique el ámbito o espacio donde estas estén siendo seteadas.

In [3]:
# Inicializando variables que serán usadas más adelante

incidencia_delictiva_link = 'https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva'
incidencia_title = ''
incidencia_description = ''

folder_path_data = '../rawdata'
folder_path_log = '../logs/descarga/'
csv_filepath = ''

# Creando variable que contendra el listado de cada source descargado.
sources_list = []

# Creando variable que contendra el listado de cada nombre de archivo descargado y que esta siendo enviado a nuestro repositorio
names_files_list = []

texto_list = ''

In [5]:
# sección de funciones para descarga de archivos desde fuentes que vienen de enlaces proporcionados por la página del gobierno.

# Función utilizada para la descarga de archivos almacenados en google drive por parte de la institución donde se requiere obtener los datos.
# - document: recibe el id del documento en google drive.
# - description: Texto de la etiqueta de enlace, el cual describe la naturaleza de la información a descargar.
def download_csv_from_http(document, description):

  print(f"link " + description)

  try:

    # Verificando si la carpeta existe, si no, crearla
    if not os.path.exists(folder_path_data):
        os.makedirs(folder_path_data)

    # Url de descarga a traves de google drive
    csv_url = f'https://drive.google.com/uc?id={document}'

    # Descargando archivos desde la url proporcionada
    response_csv = requests.get(csv_url)

    # Parte donde se hace una limpieza del parámetro description enviado,
    # esto para construir el nombre que tendrá el archivo csv que se descarga a partir de los datos de las fuentes
    # Utilizando expresión regular para reemplazar caracteres especiales y espacios por un solo guion bajo aplicándolo en la descripción del enlace.
    csv_filename = re.sub(r'[^\w\s]+', '', description)
    csv_filename = re.sub(r'\s+', '', csv_filename)
    # Utilizando unidecode para quitar los acentos y no cause conflictos como nombre de archivo
    csv_filename = unidecode(csv_filename)
    csv_filename = csv_filename + ".csv"

    # Construyendo la ruta completa del archivo
    csv_filepath = os.path.join(folder_path_data, csv_filename)

    names_files_list.append(csv_filename)
    #------------------------------------------------------------------------------------------------------------------------------------

    # Guardando el archivo descargado, en el repositorio del proyecto
    with open(csv_filepath, 'wb') as csv_file:
      csv_file.write(response_csv.content)

    # Leer el archivo CSV con la codificación original
    df = pd.read_csv(csv_filepath, encoding='latin1')

    # Guardar el archivo CSV con codificación UTF-8
    df.to_csv(csv_filepath, index=False, encoding='utf-8')

  except exception as err:
    print(f'Genero error: ' + description)

# *****************************************************************************************************************************************

# Función utilizada para la descarga de archivos almacenados en google drive por parte de la institución donde se requiere obtener los datos.
# - document: recibe el id del documento en google drive.
# - description: Texto de la etiqueta de enlace, el cual describe la naturaleza de la información a descargar.
# (Para descarga de documentos muy grandes, libreria gdown)
def download_csv_from_http_high(document, description):

  print(f"link " + description)

  try:

    # Verificando si la carpeta existe, si no, crearla
    if not os.path.exists(folder_path_data):
      os.makedirs(folder_path_data)

    # Url de descarga a traves de google drive
    csv_url = f'https://drive.google.com/uc?id={document}'

    # Parte donde se hace una limpieza del parámetro description enviado,
    # esto para construir el nombre que tendrá el archivo csv que se descarga a partir de los datos de las fuentes
    # Utilizando expresión regular para reemplazar caracteres especiales y espacios por un solo guion bajo aplicándolo en la descripción del enlace.
    csv_filename = re.sub(r'[^\w\s]+', '', description)
    csv_filename = re.sub(r'\s+', '', csv_filename)
    csv_filename = unidecode(csv_filename)
    csv_filename = csv_filename + ".csv"

    # Construyendo la ruta completa del archivo
    csv_filepath = os.path.join(folder_path_data, csv_filename)

    names_files_list.append(csv_filename)
    #----------------------------------------------------------------------------------------------------------------------------------------

    # Utilizando wget para descargar archivos grandes sin restricciones
    gdown.download(csv_url, csv_filepath, quiet=False)

    # Leer el archivo CSV con la codificación original
    df = pd.read_csv(csv_filepath, encoding='latin1')

    # Guardar el archivo CSV con codificación UTF-8
    df.to_csv(csv_filepath, index=False, encoding='utf-8')

  except exception as err:
    print(f'Genero error: ' + description)

In [6]:
# Descargando archivos proporcionados por https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva
print('# Descargando archivos proporcionados por https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva')
# Realizando Web scraping a la página donde están las distintas fuentes de datos
# Esto asegura que los datos puedan ser descargados de forma automática sin necesidad de estar descargado uno por uno directamente desde la página.
url = 'https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
pretty_soup = soup.prettify()

# Obteniendo el texto dentro de la etiqueta <title>
incidencia_source_title = soup.title.text.strip()

# Separando el texto por los pipes "|", esto porque el texto original viene con dicho carácter especial
title_parts = incidencia_source_title.split('|')

# Limpiando los espacios en blanco alrededor de cada parte del titulo encontrado
title_parts = [part.strip() for part in title_parts]

incidencia_title = title_parts[0] + " - " + title_parts[1]

# Buscando todas las etiquetas <h2> y extraer su texto, esto ayudara a poder extraer la descripción de la página, la explicación de la razón de la página.
h2_tags = soup.find_all('h2')

# Se recorren cada una de los parrafos H2 encontrados para luego extraer el texto en donde comience con "En esta pagina...".
for h2_tag in h2_tags:
    text_inside_h2 = h2_tag.text.strip()

    if(text_inside_h2.find('En esta página')!=-1) :
      incidencia_description = text_inside_h2

# Buscando todas las etiquetas <ul>, esto ayudara más adelante a poder realizar las descargas, pues aquí se encontrará las url de cada enlace.
ul_tags = soup.find_all('ul')

for ul_tag in ul_tags:
    # Buscando todas las etiquetas <li> dentro de cada <ul>
    li_tags = ul_tag.find_all('li')

    for li_tag in li_tags:
         # Buscando todas las etiquetas <a> dentro de cada <li> con texto que comienza con "Cifras"
        a_tags = li_tag.find_all('a', href=True, string=lambda x: x and x.startswith('Cifras'))

        for a_tag in a_tags:
            # Obteniendo el valor del atributo href, para extraer su url
            href_value = a_tag['href']

            # Utilizando expresión regular para extraer la parte antes de "/view" de cada url esto para conseguir el id del documento
            match = re.search(r'/file/d/(.*?)/view', href_value)
            # match = re.search(r'(.*?)/view', href_value)

            # En caso de encontrar coicidencia, se procede a extraer el id del documento de descarga.
            if match:
              extracted_part = match.group(1)
              if a_tag.text.find(str(datetime.now().year)) != -1:
                # Condicionante para poder detectar si la descarga es de datos referentes a municipios, esto por conocimiento, son descargas más pesadas y por lo cual se ocupa de librería especial.
                if a_tag.text.find('Municipal') != -1:
                  download_csv_from_http_high(extracted_part, a_tag.text)

                  # Imprimiendo el tag descriptivo y la url para mostrarla en pantalla
                  print(f""+ a_tag.text + ": " + href_value)
                else:
                  download_csv_from_http(extracted_part, a_tag.text)

                #Creando listado que será usado para el documento con la descripción de la descarga de las distintas fuentes, de incidencia delictiva
                sources_list.append(f"- {a_tag.text}: {href_value}")

# Descargando archivos proporcionados por https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva
link Cifras de Incidencia Delictiva Estatal, 2015 - agosto  2023
link Cifras de Incidencia Delictiva Municipal, 2015 - agosto 2023. 


Downloading...
From (uriginal): https://drive.google.com/uc?id=13TjyJ9RkR49o0eWTFvhNqazeL4maORYp
From (redirected): https://drive.google.com/uc?id=13TjyJ9RkR49o0eWTFvhNqazeL4maORYp&confirm=t&uuid=570dd9a5-00cc-498b-bad7-518189d2659f
To: c:\Proyectos\Maestria\IngenieriaCaracteristicas\Proyectos\IC-Proyecto-Indicadores-Delictivos\rawdata\CifrasdeIncidenciaDelictivaMunicipal2015agosto2023.csv
100%|██████████| 306M/306M [00:42<00:00, 7.26MB/s] 


Cifras de Incidencia Delictiva Municipal, 2015 - agosto 2023. : https://drive.google.com/file/d/13TjyJ9RkR49o0eWTFvhNqazeL4maORYp/view?usp=sharing
link Cifras de Víctimas del Fuero Común, 2015 - agosto 2023
link Cifras de Incidencia Delictiva Federal, 2012 - agosto 2023


In [7]:
# Convertiendo la lista de nombres de archivos generados a DataFrame
df_names = pd.DataFrame(names_files_list)

df_names = df_names.rename(columns={0: "nombre"})

# Guardando el DataFrame como un archivo CSV
df_names.to_csv(f'{folder_path_data}/nombres_dataset.csv', index=False, encoding='utf-8')

In [8]:
# Leyendo archivos en DataFrames, a partir de los archivos csv generados, provenientes de las fuentes de datos.
df_incidencia = pd.read_csv(f'{folder_path_data}/CifrasdeIncidenciaDelictivaEstatal2015agosto2023.csv')

print(df_incidencia.shape)

(28224, 19)


In [9]:
# Creando el archivo de texto con descripción general de las fuentes de descargar y parte del proceso ETL

# Verificando si la carpeta existe, si no, crearla
if not os.path.exists(folder_path_log):
  os.makedirs(folder_path_log)

# Uniendo elementos de la lista de las fuentes en donde se han descargado los datos (sources_list)
texto_list = "\n".join(sources_list)

date_now = datetime.now()

# Variable que contiene el texto que tendrá el archivo txt que se generará y descargará.
description = f"""
Proyecto: contando una historia a través de los datos (análisis de la variabilidad del comportamiento delictivo
en relación con factores temporales y geográficos, y su correlación con el desarrollo social y urbano para la prevención del delito)

Equipo participante:
- Ernesto
- Luis Andrés
- Mario Estrada Ferreira

Descripción de los datos descargados:
- Las fuentes fueron descargadas y donde se puede encontrar información adicional en el sitio web: {incidencia_delictiva_link}

Fuentes:
{texto_list}

Fechas de descarga:
- El proceso de descarga de los datos se realizó el: {date_now.strftime('%Y-%m-%d %H:%M:%S')}
"""
# Se crea el archivo a partir del texto de arriba, el cual se guardará con el nombre descriptivo más la fecha día*mes*año, para identificar la descarga por día.
with open(f'{folder_path_log}/fuentes_descripcion{date_now.strftime("%d%m%y")}.txt', 'w') as description_file:
    description_file.write(description)

print("Archivos de datos descargados y descripción en txt creada.")

Archivos de datos descargados y descripción en txt creada.


In [10]:
nombres_dataset = pd.read_csv(f'{folder_path_data}/nombres_dataset.csv')
df_nombres_dataset = pd.DataFrame(nombres_dataset)
df_nombres_dataset.head()

,nombre
0,CifrasdeIncidenciaDelictivaEstatal2015agosto20...
1,CifrasdeIncidenciaDelictivaMunicipal2015agosto...
2,CifrasdeVictimasdelFueroComun2015agosto2023.csv
3,CifrasdeIncidenciaDelictivaFederal2012agosto20...


In [11]:
# Leyendo archivos en DataFrames, a partir de los archivos csv generados, provenientes de las fuentes de datos.
df_incidencia = pd.read_csv(f'{folder_path_data}/CifrasdeIncidenciaDelictivaEstatal2015agosto2023.csv')

print(df_incidencia.head())

    Año  Clave_Ent         Entidad            Bien jurídico afectado  \
0  2015          1  Aguascalientes  La vida y la Integridad corporal   
1  2015          1  Aguascalientes  La vida y la Integridad corporal   
2  2015          1  Aguascalientes  La vida y la Integridad corporal   
3  2015          1  Aguascalientes  La vida y la Integridad corporal   
4  2015          1  Aguascalientes  La vida y la Integridad corporal   

  Tipo de delito  Subtipo de delito          Modalidad  Enero  Febrero  Marzo  \
0      Homicidio   Homicidio doloso  Con arma de fuego      3        0      2   
1      Homicidio   Homicidio doloso    Con arma blanca      1        1      0   
2      Homicidio   Homicidio doloso  Con otro elemento      0        0      2   
3      Homicidio   Homicidio doloso    No especificado      2        0      0   
4      Homicidio  Homicidio culposo  Con arma de fuego      0        0      0   

   Abril  Mayo  Junio  Julio  Agosto  Septiembre  Octubre  Noviembre  \
0      1

In [12]:
# Leyendo archivos en DataFrames, a partir de los archivos csv generados, provenientes de las fuentes de datos.
df_incidencia = pd.read_csv(f'{folder_path_data}/CifrasdeIncidenciaDelictivaFederal2012agosto2023.csv')

print(df_incidencia.head())

    AÑO  INEGI         ENTIDAD                   LEY         CONCEPTO  \
0  2012    1.0  AGUASCALIENTES  CODIGO PENAL FEDERAL  CONTRA LA SALUD   
1  2012    1.0  AGUASCALIENTES  CODIGO PENAL FEDERAL  CONTRA LA SALUD   
2  2012    1.0  AGUASCALIENTES  CODIGO PENAL FEDERAL  CONTRA LA SALUD   
3  2012    1.0  AGUASCALIENTES  CODIGO PENAL FEDERAL  CONTRA LA SALUD   
4  2012    1.0  AGUASCALIENTES  CODIGO PENAL FEDERAL  CONTRA LA SALUD   

         TIPO  ENERO  FEBRERO  MARZO  ABRIL  MAYO  JUNIO  JULIO  AGOSTO  \
0  PRODUCCION      0        0      0      0     0      0      0       0   
1  TRANSPORTE      0        0      0      3     1      3      0       0   
2     TRAFICO      0        0      0      0     0      0      0       0   
3    COMERCIO     13        9     18     16     8     14     13       6   
4  SUMINISTRO      0        0      0      0     0      0      0       0   

   SEPTIEMBRE  OCTUBRE  NOVIEMBRE  DICIEMBRE  
0         0.0      0.0        1.0        0.0  
1         0.0   

In [13]:
# Leyendo archivos en DataFrames, a partir de los archivos csv generados, provenientes de las fuentes de datos.
df_incidencia = pd.read_csv(f'{folder_path_data}/CifrasdeIncidenciaDelictivaMunicipal2015agosto2023.csv')

print(df_incidencia.head())

    Año  Clave_Ent         Entidad  Cve. Municipio       Municipio  \
0  2015          1  Aguascalientes            1001  Aguascalientes   
1  2015          1  Aguascalientes            1001  Aguascalientes   
2  2015          1  Aguascalientes            1001  Aguascalientes   
3  2015          1  Aguascalientes            1001  Aguascalientes   
4  2015          1  Aguascalientes            1001  Aguascalientes   

             Bien jurídico afectado Tipo de delito  Subtipo de delito  \
0  La vida y la Integridad corporal      Homicidio   Homicidio doloso   
1  La vida y la Integridad corporal      Homicidio   Homicidio doloso   
2  La vida y la Integridad corporal      Homicidio   Homicidio doloso   
3  La vida y la Integridad corporal      Homicidio   Homicidio doloso   
4  La vida y la Integridad corporal      Homicidio  Homicidio culposo   

           Modalidad  Enero  ...  Marzo  Abril  Mayo  Junio  Julio  Agosto  \
0  Con arma de fuego      2  ...      1      1     0      1   

In [14]:
# Leyendo archivos en DataFrames, a partir de los archivos csv generados, provenientes de las fuentes de datos.
df_incidencia = pd.read_csv(f'{folder_path_data}/CifrasdeVictimasdelFueroComun2015agosto2023.csv')

print(df_incidencia.head())

    Año  Clave_Ent         Entidad            Bien jurídico afectado  \
0  2015          1  Aguascalientes  La vida y la Integridad corporal   
1  2015          1  Aguascalientes  La vida y la Integridad corporal   
2  2015          1  Aguascalientes  La vida y la Integridad corporal   
3  2015          1  Aguascalientes  La vida y la Integridad corporal   
4  2015          1  Aguascalientes  La vida y la Integridad corporal   

  Tipo de delito  Subtipo de delito          Modalidad   Sexo  \
0      Homicidio   Homicidio doloso  Con arma de fuego  Mujer   
1      Homicidio   Homicidio doloso    Con arma blanca  Mujer   
2      Homicidio   Homicidio doloso  Con otro elemento  Mujer   
3      Homicidio   Homicidio doloso    No especificado  Mujer   
4      Homicidio  Homicidio culposo  Con arma de fuego  Mujer   

            Rango de edad  Enero  ...  Marzo  Abril  Mayo  Junio  Julio  \
0  Menores de edad (0-17)      0  ...      0      0     0      0      0   
1  Menores de edad (0-17) 